### Assignment #2: Face recognition
 The program will process the images from a zip file provided in the
 `zip` folder that contains the pages of a newspaper, it will process
 the text in the image using `pytesseract` and will make a sheet
 with the faces that match certain string in that page

In [2]:
from zipfile import ZipFile
from PIL import Image, ImageFont, ImageDraw
import cv2 as cv
import glob
import pytesseract

# Extract zip
with ZipFile("zip/small_img.zip", "r") as object_zip:
    object_zip.extractall('temp')

# Iterate through the images in the zip
png_counter = len(glob.glob1('temp', "*.png"))

# Face cascade
face_cascade = cv.CascadeClassifier("patterns/haarcascade_frontalface_default.xml")

# Font
font = ImageFont.truetype("fonts/fanwood-webfont.ttf", 18)

# Images
images = []

def get_results(user_input):
    for i in range(png_counter):
        file_path = "temp/a-"+str(i)+".png"
        img = Image.open(file_path)
        grey = cv.cvtColor(cv.imread(file_path), cv.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(grey, scaleFactor=1.2, minNeighbors=5)

        # Append a tuple that contains
        # Numpy arrays of original and grey image
        # The text that was extracted from the article
        # Faces detected

        pil_image = Image.fromarray(grey, "L")
        txt = pytesseract.image_to_string(pil_image)
        images.append((img, grey, txt, faces, file_path))

    for i, image in enumerate(images):
        # If user input's matches with something in the text
        if user_input in image[2]:
            # Face detection if goes after detecting if a given text
            # exists in the extracted text (i.e, "Mark")

            n_faces = len(image[3])
            rows = (n_faces // 5)+1

            # Position in sheet
            pos_x = 0
            pos_y = 30
            index = 0

            # Initial values
            size = 128

            sheet = Image.new(image[0].mode, (5*128, (rows*128)+30))
            canvas = ImageDraw.Draw(sheet)
            canvas.rectangle([(sheet.width, 0), (0, 30)], fill=(255, 255, 255))
            canvas.text((0, 0), "Results found in file {}".format(image[4]), font=font, fill=(0,0,0))

            if n_faces == 0:
                canvas.rectangle([(sheet.width, 30), (0, 60)], fill=(255, 255, 255))
                canvas.text((0, 30), "But there were no faces in that file!", font=font, fill=(0,0,0))

            for x, y, w, h in image[3]:
                cropped = image[0].crop((x, y, x+w, y+h))
                cropped.thumbnail((size, size), Image.ANTIALIAS)
                sheet.paste(cropped, (pos_x, pos_y))

                if index == 4:
                    pos_y += size
                    pos_x = 0
                    index = 0
                else:
                    pos_x += size
                    index += 1

            save_path = 'images/A2/sheet-'+str(i)+'.png'
            sheet.save(save_path)
            sheet.show()

str_match = input("Please enter a string to match with the image")
get_results(str_match)